In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests as req
from census import Census
import json
import csv



# Census API Key
#from config import api_key
c = Census("49f2db3e7faca8ecd92103d5bd5c5c765442d598", year=2016)


In [2]:
# Logic:
#Get the cites to query
#Get the census fields to query

sourceFile = pd.ExcelFile('Project1_AmazonSites.xlsx')

SitesDF=sourceFile.parse('AmazonSites') 

CitiesDf=sourceFile.parse('AmazonCities') 
CitiesDf=CitiesDf[['PlaceCode','StateCode']]

CensusFieldsDF=sourceFile.parse('CensusFields')
CensusFieldsDF = CensusFieldsDF[["Datafield","CensusCode"]]

#Converting the CensusFieldsDF into a dictionary.
dictCensusFields = CensusFieldsDF.set_index('CensusCode').to_dict()
dictCensusFields = dictCensusFields['Datafield']

print(dictCensusFields)
#CitiesDf.head()

{'B01002_001E': 'Median Age', 'B19013_001E': 'Median Household Income', 'B25077_001E': 'Median Home Value', 'B25064_001E': 'Median Rent', 'B23025_002E': 'LaborForce', 'B23025_004E': 'Emp_Total', 'B23025_003E': 'Emp_civilian_labor_force', 'C24010_044E': 'Emp_Comp_Math_Female', 'C24010_040E': 'Emp_Mgment_Occp_Female', 'C24010_008E': 'Emp_Comp_Math_Female', 'C24010_006E': 'Emp_Mgment_Occp_Male', 'B15003_021E': 'Ed_Associates', 'B15003_022E': 'Ed_Bachelors', 'B15003_025E': 'Ed_Doctorate', 'B15003_018E': 'Ed_GED', 'B15003_017E': 'Ed_HighSchool', 'B15003_023E': 'Ed_Masters', 'B15003_024E': 'Ed_Professional', 'B01003_001E': 'Total Population', 'B02001_004E': 'Pop_American_Indian', 'B02001_005E': 'Population_Asian', 'B02001_003E': 'Pop_Black', 'B03001_003E': 'Pop_Hispanic_Origin', 'B02001_006E': 'Pop_Native_Hawaiian', 'B02001_007E': 'Pop_Other', 'B02001_008E': 'Pop_two_or_more_races', 'B02001_002E': 'Pop_White', 'B08136_001E': 'Commute_Time', 'B08136_004E': 'Commute_Time_Carpool', 'B08136_012E

In [4]:

# Build your Census API call.
arrCensusFields = ['NAME']

for k in dictCensusFields.keys():
    arrCensusFields.append(k)
    
censusDF= pd.DataFrame()

for city in CitiesDf.values:
    cityCode = str(city[0])
    stateCode = str(city[1])

    if len(cityCode)==4:
        cityCode = "0"+cityCode
    if len(stateCode)==1:
        stateCode = "0"+stateCode
        
    varGeo = {'for':'place:'+ cityCode,'in':'state:'+stateCode}
    temp1 = c.acs5.get(arrCensusFields, varGeo)
  
    tempDF = pd.DataFrame(temp1[0], index = [0])    
    censusDF = censusDF.append(tempDF,ignore_index = True)


censusDF = censusDF.rename(columns=dictCensusFields)
censusDF.to_csv("Cities_Demographics.csv")



,Median Age,Total Population,Pop_White,Pop_Black,Pop_American_Indian,Population_Asian,Pop_Native_Hawaiian,Pop_Other,Pop_two_or_more_races,Pop_Hispanic_Origin,...,Median Household Income,LaborForce,Emp_civilian_labor_force,Emp_Total,Median Rent,Median Home Value,Emp_Mgment_Occp_Male,Emp_Comp_Math_Female,Emp_Mgment_Occp_Female,Emp_Comp_Math_Female
count,8.000000,8.000000e+00,8.000000e+00,8.000000e+00,8.00000,8.000000e+00,8.000000,8.000000e+00,8.000000,8.000000e+00,...,8.000000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.00000,8.000000,8.000000,8.000000
mean,33.637500,2.277203e+06,1.096386e+06,5.223889e+05,9907.25000,2.454939e+05,1665.375000,3.281315e+05,73229.750000,7.163308e+05,...,57199.000000,1.215056e+06,1.213957e+06,1.106664e+06,1165.375000,357250.000000,32574.00000,24513.625000,85163.750000,8700.000000
std,1.358505,2.801473e+06,1.219406e+06,6.674602e+05,12715.72402,3.972234e+05,2405.776616,4.863364e+05,93120.176107,9.497127e+05,...,7635.658452,1.438716e+06,1.438388e+06,1.311259e+06,171.337543,138209.085912,35141.66021,22341.358489,91226.775925,7803.957988
min,31.700000,4.413260e+05,1.827940e+05,6.865100e+04,1308.00000,1.823500e+04,122.000000,4.802000e+03,10032.000000,2.183800e+04,...,49398.000000,2.458730e+05,2.455750e+05,2.200440e+05,966.000000,218200.000000,7307.00000,6139.000000,21643.000000,2633.000000
25%,32.775000,6.078038e+05,2.653180e+05,1.573885e+05,1984.75000,2.310175e+04,253.750000,2.657975e+04,16433.250000,6.376025e+04,...,51262.000000,3.516608e+05,3.489380e+05,3.201352e+05,995.250000,224475.000000,13336.50000,10058.500000,30599.500000,3524.500000
50%,33.650000,7.833940e+05,5.187815e+05,2.788015e+05,3246.50000,6.170800e+04,555.500000,5.033900e+04,30555.000000,2.189545e+05,...,56853.500000,4.606575e+05,4.600830e+05,4.314310e+05,1173.500000,340500.000000,15146.00000,17893.000000,44431.500000,7024.500000
75%,34.175000,3.015231e+06,1.504799e+06,4.747465e+05,12528.50000,2.371350e+05,1678.250000,4.540550e+05,86161.500000,1.069305e+06,...,59215.500000,1.611831e+06,1.611250e+06,1.444081e+06,1311.000000,498750.000000,42924.25000,29537.250000,107179.750000,9588.250000
max,35.900000,8.461961e+06,3.644982e+06,2.064927e+06,33388.00000,1.159883e+06,6716.000000,1.277265e+06,277521.000000,2.457945e+06,...,72935.000000,4.369307e+06,4.367089e+06,3.991617e+06,1369.000000,508900.000000,112186.00000,74631.000000,291733.000000,26761.000000
